## **Initialize Pinecone**

In [3]:
import pinecone

pinecone.init(api_key="87d21a17-cc6f-4e2d-a4fe-2e5a86ee6fa8", environment="us-west4-gcp-free")

In [4]:
index = pinecone.GRPCIndex("amazon-products")

In [6]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## **Upsert Index**

**Note**: Skip this step if you only want to query the index(semantic search)

In [28]:
import pickle

model_name = "gte_base"
with open(f"embeddings_product_titles_{model_name}.pkl", "rb") as fIn:
    stored_data = pickle.load(fIn)
    asin = stored_data["asin"]
    titles = stored_data["product_titles"]
    embeddings = stored_data["embeddings"]

In [29]:
embeddings = embeddings.tolist()

In [40]:
vectors = list(zip(asin, embeddings))
print("Number of vectors:", len(vectors))

Number of vectors: 91562


In [39]:
index.upsert(vectors, batch_size=64)

Upserted vectors:   0%|          | 0/91562 [00:00<?, ?it/s]

upserted_count: 91562

## **Query Index**

Load the dataset in order to later extrart the product title

In [ ]:
from pprint import pprint

import delta_sharing

profile_file = "config.share"
client = delta_sharing.SharingClient(profile_file)
print("Available tables:")
pprint(client.list_all_tables())

# load metadata table
table_url = profile_file + "#share__products.silver.amazon_metadata_silver_selected"
df = delta_sharing.load_as_pandas(table_url)

In [42]:
import torch
from sentence_transformers import SentenceTransformer

device = "cuda" if torch.cuda.is_available() else "cpu"

model = SentenceTransformer("thenlper/gte-base", device=device)

No sentence-transformers model found with name /home/studio-lab-user/.cache/torch/sentence_transformers/thenlper_gte-base. Creating a new one with MEAN pooling.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [64]:
query = "microsoft excel"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(xq, top_k=5)
for match in xc["matches"]:
    id = match["id"]
    print(df.query("asin == @id")["title"].item())

Microsoft Excel 2010
Excel @ Mathematics
Microsoft Excel Mac 2008
Microsoft Excel for Students & Teachers 2003 [Old Version]
Microsoft Office Excel Home and Student 2007 [Old Version]
